# 2-1 三种计算图

一般构建计算图的方式有三种：
* 静态计算图
* 动态计算图
* Autograph

## 1 计算图特点分析

1. 在TensorFlow1.x时代, 采用的是**静态图**, 也就是先需要通过TensorFlow的各种算子构建出图的结构, 然后开启一个Session去执行计算。
2. 在TensorFlow2.x时代, 采用的是**动态图**, 也就是使用一个算子之后, 该算子会被动态地加入到计算途中立即执行得到结果。
3. 使用动态图, 即Eager Excution的好处就在于方便调试程序, 它让TensorFlow程序与普通的Python变得一模一样。但是, 使用动态图会有多次Python程序和TensorFlow的C++进程之间的通信导致效率低下。
4. 如果需要在TensorFlow2.0中使用静态图, 可以使用@tf.function装饰器将普通Python函数转换成对应的TensorFlow计算图构建代码。运行该函数就相当于在TensorFlow1.0中执行Session。而使用@tf.function构建静态图的方式叫做**Autograph**

## 2 计算图简介
计算图由节点和线组成

节点代表操作符Operator, 线表示计算间的依赖

实现表示数据传递依赖, 传递的数据就是张量

虚线通常表示控制依赖, 即执行的先后顺序

## 3 静态计算图构建

In [1]:
import tensorflow as tf

In [4]:
# 利用TensorFlow2中compat.v1模块调用Tensorflow1中的静态图
g = tf.compat.v1.Graph()
with g.as_default():
    x = tf.compat.v1.placeholder(name = "x", shape = [], dtype = tf.string)
    y = tf.compat.v1.placeholder(name = "y", shape = [], dtype = tf.string)
    z = tf.strings.join([x, y], name = "join", separator = " ")

with tf.compat.v1.Session(graph = g) as sess:
    result = sess.run(fetches = z, feed_dict = {x: "hello", y: "world"})
    print(result)

b'hello world'


## 4 构建动态图

In [6]:
x = tf.constant("hello")
y = tf.constant("world")
z = tf.strings.join([x, y], separator = " ")
print(z)
tf.print(z)

tf.Tensor(b'hello world', shape=(), dtype=string)
hello world


## 5 TensorFlow2中的Autograph
由于动态图运行效率较低, 可以使用@tf.function将普通的Python函数转换为静态图的构建代码
在TensorFlow1中使用计算图分为两步：第一步是定义计算图, 第二步是执行计算图
在TensorFlow2中采用Autograph的方式, 第一步定义计算图变成了定义函数, 第二步执行计算图变成了调用函数

In [8]:
@tf.function
def strjoin(x, y):
    return tf.strings.join([x, y], separator = " ")

print(strjoin(tf.constant("hello"), tf.constant("world")))

tf.Tensor(b'hello world', shape=(), dtype=string)


In [10]:
logdir = "2-2_autograph_log"
writer = tf.summary.create_file_writer(logdir)
# 开启autograph跟踪
tf.summary.trace_on(graph = True, profiler = True)
# 执行autograph
res = strjoin("hello", "world")

# 把图信息写入日志
with writer.as_default():
    tf.summary.trace_export(
        name = "autograph",
        step = 0,
        profiler_outdir = logdir
    )
    
from tensorboard import notebook
notebook.start('--logdir ' + logdir)